# Run Q3Dfit from this Jupyter Notebook in Astroconda Environment.

This Jupyter notebook allows you to run Q3Dfit, a PSF decomposition and spectral analysis package tailored for JWST NIRSpec and MIRI IFU observations. 

Q3Dfit is developed as a science-enabling data product by the Early Release Science Team #1335 Q3D. You can find more information about this ERS program **Q3D** [here](https://wwwstaff.ari.uni-heidelberg.de/dwylezalek/q3d.html) and [here](https://www.stsci.edu/jwst/science-execution/approved-programs/dd-ers/program-1335).

The software is based on the existing package IFSFIT developed by Dave Rupke (see [ADS link](https://ui.adsabs.harvard.edu/abs/2017ApJ...850...40R/abstract)).

The following notebook will guide you through the initialization procedure and will then perform the analysis. 

## Table of Contents

* [1. Initialization](#chapter1)
    * [1.1. Setting up the directory tree](#chapter1_1)
    * [1.2. Setting up the fitting parameters](#chapter1_2)        
* [2. Run Q3Dfit](#chapter2)
* [3. Run plotting script (q3da)](#chapter3)


<font color='red'>We need to decide where the makeqsotemplate functionality goes. </font>

## 1. Initialization <a class="anchor" id="chapter1"></a>

In [ ]:
import os.path
import numpy as np
from q3dfit.common import questfit_readcf

### 1.1. Setting up the directory tree <a class="anchor" id="chapter1_1"></a>

Define the directories in which your data cube(s) that you want to analyse are stored and the output directories. We recommend creating a working directory that you name after your target, in which all outputs from Q3Dfit will be saved. 

In [ ]:
# Input files
infile = '../test/test_questfit/22128896_mock_cube.fits'

# Output directories/files
outdir = ''
logfile = outdir+'test_questfit_fitlog.txt'
path_to_npy_file = outdir # path for initproc.npy

In [ ]:
outdir = '/Users/drupke/specfits/q3dfit/testing/22128896/'
logfile = outdir+'test_questfit_fitlog.txt'
path_to_npy_file = outdir # path for initproc.npy



Some general information about your cube:

In [ ]:
ncols = 1 # number of columns
nrows = 1 # number of rows
platescale = 0.3

fitrange = np.array([5.422479152679443, 29.980998992919922])  # micron

if not os.path.isfile(infile): print('Data cube not found.')

In [ ]:
### more MIR settings
#   These are unique to the user
global_ice_model = 'ice_hc'
global_ext_model = 'CHIAR06'
cffilename = '../test/test_questfit/22128896.cf'
config_file = questfit_readcf.readcf(cffilename)

### 1.2. Setting up the fitting parameters <a class="anchor" id="chapter1_2"></a>

What lines do you want to fit? You can choose from the linelist <font color='red'>provide reference here</font>.

In [ ]:
lines = ['[ArII]6.99', '[ArIII]8.99', '[NeII]12.81', 
         '[NeIII]15.56', 'H2_00_S1', '[SIII]18.71']
# 'H2_00_S5', 'H2_00_S3'

How many components do you want to be fitted to the emission lines?

In [ ]:
maxncomp = 1

In [ ]:
# Initialize line ties, n_comps, z_inits, and sig_inits.
linetie = dict()
ncomp = dict()
zinit_gas = dict()
siginit_gas = dict()
for i in lines:
    linetie[i] = '[NeII]12.81'
    ncomp[i] = np.full((ncols,nrows),maxncomp)
    zinit_gas[i] = np.full((ncols,nrows,maxncomp),0.)
    siginit_gas[i] = np.full(maxncomp, 1000.) #0.1) #1000.)
    zinit_stars=np.full((ncols,nrows),0.0)

In [ ]:
siglim_gas = np.array([5., 4000.])

One dictionary must be defined for a line plot to appear. `argspltlin1` holds the options for the first line plot, and `argspltlin2` is for the second. Parameter options are fed in as keyword/data pairs.

Required keywords in each dictionary:
* `nx`: Number of subplots in the horizontal direction
* `ny`: Number of subplots in the vertical direction
* Options for centerting the plot:
    - `line`: a string list of line labels
    - `center_obs`: a float list of wavelengths of each subplot center, in the observed (plotted) frame
    - `center_rest`: a float list of wavelengths of each subplot center, in the rest frame, which are converted to obs. frame

Optional keywords:
* `size`: float list of widths in wavelength space of each subplot; if not specified, default is 300 $Å$
* `IR`: set to `True` to use infrared-style plot

In [ ]:
argspltlin1 = {'nx': 3,
               'ny': 2,
               'line': lines,
               'size': [3., 3., 3., 3., 3., 3.],
               'IR': True}

The following cell will create the initialisation structure. **Do not change anything in this cell.** 

In [ ]:
initproc = { \
            # Required pars
            'fcninitpar': 'parinit', #'gmos',
            'fitran': fitrange,
            'fluxunits': 1,
            'infile': infile,
            'label':
                config_file['source'][0].replace('.ideos','').replace('.npy', ''),
            'lines': lines,
            'linetie': linetie,
            'maxncomp': maxncomp,
            'name': 'IRAS21219m1757_dlw_qst',
            'ncomp': ncomp,
            'outdir': outdir,
            'zinit_stars': zinit_stars,
            'zinit_gas': zinit_gas,
            'zsys_gas': 0.0,
            # Optional pars
            'argscontfit': {'config_file': cffilename,
                            'global_ice_model': global_ice_model,
                            'global_ext_model': global_ext_model,
                            'models_dictionary': {},
                            'template_dictionary': {}},
            'argslinelist': {'vacuum': False},
            'argspltlin1': argspltlin1,
            'fcncheckcomp': 'checkcomp',
            'fcncontfit': 'questfit',
            'maskwidths_def': 1000,
            'emlsigcut': 2,
            'logfile': logfile,
            'siglim_gas': siglim_gas,
            'siginit_gas': siginit_gas,
            'siginit_stars': 100,
            'nocvdf': 1,
            'waveext': 4,
            'datext': 1,
            'varext': 2,
            'dqext': 3,
            'zerodq': True,
            'argsreadcube': {'fluxunit_in': 'Jy',
                            'waveunit_in': 'angstrom',
                            'waveunit_out': 'micron'},          
            'plotMIR': True,
        }

If you want to run `q3dfit` in batch mode, run this cell, which saves initproc to an `npy` file. In your python command line, read in that `npy` file with
<pre><code>import numpy as np
initproc = np.load('path/to/the/npy/file/initproc.npy',allow_pickle=True)</code></pre>
and then run `q3dfit` with 
<pre><code>from q3dfit.common.q3df
import q3df q3df(initproc[()],cols=cols,rows=rows)</code></pre>

In [ ]:
initproc_npy = 'initproc.npy'
np.save(path_to_npy_file+initproc_npy,initproc)

## 2. Run Q3Dfit <a class="anchor" id="chapter2"></a>

In [ ]:
# This may be unique to the user, insert your path to the q3dfit/ folder here
import sys
if '../../' not in sys.path:
    sys.path.append('../../')

from q3dfit.common.q3df import q3df
from q3dfit.common.q3da import q3da

In [ ]:
cols = [1]
rows = [1]

In [ ]:
q3df(initproc, cols=cols, rows=rows, quiet=False)

## 3. Run plotting script (q3da) <a class="anchor" id="chapter3"></a>

In [ ]:
%matplotlib inline
q3da(initproc, cols=cols, rows=rows, quiet=False)

In [ ]:
%matplotlib
q3da(initproc, cols=cols, rows=rows, quiet=False)